# POS Tagging using Viterbi Algorithm

### Dr. Uzair Ahmad
2020.12.23

The Viterbi Algorithm is a dynamic programming algorithm for finding the most likely sequence of hidden states, in the case of POS tagging, the hidden states are the parts of speech (POS). The algorithm requires two main inputs, which are typically expressed in matrix form:

- **Transition probabilities**, which provide the likelihood of moving from one hidden state to another.
- **Emission probabilities**, which provide the likelihood of an observed value given a specific hidden state.

The Viterbi Algorithm uses these probabilities to compute the most likely sequence of hidden states that lead to the observed data.

The algorithm essentially consists of two steps, a forward step (or recursion) and a backward step (or backtrace).

1. **Forward step**: It goes through the sequence from start to end, storing at each position the maximum probability of each state and the state that preceded it.
2. **Backward step**: It backtracks from the end of the sequence to the beginning using the stored information to find the most probable path.

Below is an implementation of the Viterbi Algorithm in Python:


In [52]:
def viterbi(words, tags, trans_p, emit_p):
    """
    Parameters:
    words : list of observations (e.g., words)
    tags : list of tags (e.g., POS tags)
    trans_p : transition probability matrix
    emit_p : emission probability matrix

    Return:
    The best path with its corresponding probability.
    """

    # Initialization
    # List of Viterby variables/  the maximum probability tag for a given word
    V = [{}, {}] # first element is just to keep the index start from 1
    # Dictionary of backward pointers.
    # The key in this dictionary is the state, and the value is the optimal path leading to this state.
    B = [{}, {}]
    # M : count of words
    M = len(words) - 1 # excluding 'End' tag
    # Step 1. Initial probability = start probability x emission probability
    print( f'\n##### Initialization #####\n' )
    print(f"word {1}: {words[1]}\n")
    V[1]['word'] = words[1]
    for tag in tags:
        V[1][tag] =  trans_p['sos'][tag] * emit_p[tag][words[1]]
        print(f"\t current_tag: {tag}, \
                                trans_p['sos'][{tag}] {trans_p['sos'][tag]}, \
                                emit_p[{tag}][{words[1]}] {emit_p[tag][words[1]]}, \
                                V[1][{tag}]: {V[1][tag]}")
        B[1] = 'sos'
    # Step 2. Forward recursion
    print( f'\n##### Recursion #####\n' )
    for m in range(2, M):
        V.append({})
        B.append({})
        V[m]['word'] = words[m]
        '''
        In each iteration of the forward pass, for each state,
        it computes the maximum probability of ending up in that state (considering all possible paths)
        and keeps track of the path that led to this maximum probability.
        Therefore, the path to each state is updated at each step, meaning that by the time the forward pass is finished,
        you already have the optimal path to the final state stored.
        '''
        print( f"word {m}: {words[m]}")
        for current_tag in tags:
            print(f"\t current_tag: {current_tag}, emit_p {emit_p[current_tag][words[m]]}")
            # Maximum transition probability x corresponding emission probability
            '''
            The key point here is that the Viterbi algorithm doesn't decide on
            the tag of a word based only on the emission probability for that word.
            It also takes into account the transition probabilities
            from the previous tag and the path that leads to
            the highest probability up to that point.
            '''
            p = []
            for previous_tag in tags:
              V_emit_transit =  V[m-1][previous_tag] * \
                                trans_p[previous_tag][current_tag] * \
                                emit_p[current_tag][words[m]]

              print( f"\t previous_tag: {previous_tag} \
                          trans_p[{previous_tag}]->[{current_tag}]: {trans_p[previous_tag][current_tag]} \
                          V[{m-1}][{previous_tag}]: {V[m-1][previous_tag]} \
                          V[{m}][{previous_tag}]:: {V_emit_transit}"
                  )

              p.append((V_emit_transit, previous_tag))

            max_prob, max_prob_previous_tag = max(p)
            print(f"\t max_prob_previous_tag, {max_prob_previous_tag}, max_prob: {max_prob}")
            ''' # alternate
            (max_prob, max_prob_previous_tag) = max(( V[m-1][previous_tag] * trans_p[previous_tag][current_tag] * emit_p[current_tag][words[m]] , previous_tag) for previous_tag in tags)
            '''
            # the mth Viterbi variable value
            V[m][current_tag] = max_prob

            print(f"\t max V[{m}][{current_tag}] : {V[m][current_tag]}\n")
            '''
            Build the optimal path to the current tag by appending
            the current tag to the path of the optimal previous tag.
            '''
            B[m] = max_prob_previous_tag

    # Step 3. Terminations: Maximum probability for final tag
    print( f'\n#####Termination#####\n' )
    (max_prob, final_tag) = max((trans_p[previous_tag]['eos'] * V[M-1][previous_tag], previous_tag) for previous_tag in tags)
    print(f'Termination {final_tag} {max_prob}')
    B.append({})
    B[-1] = final_tag

    # Step 4. Backward step (find the most probable path)
    print( f'\n##### Backtracking #####\n' )
    best_path = []

    for m in range(len(words) - 1, -1, -1):  # Backtrack from the last word to the first
        best_path.insert(0, [B[m]])      # Insert the current tag at the beginning of the path

    # Now 'best_path' contains the most probable sequence of tags
    print('------------------------\\n',max_prob, best_path)

    return (max_prob, best_path)


In the above code, obs represents the observed states (e.g., words in a sentence), states are the hidden states (e.g., POS tags), start_p are the start probabilities (the probability of a tag appearing at the beginning of a sentence), trans_p are the transition probabilities (the probability of moving from one tag to another), and emit_p are the emission probabilities (the probability of a word given a tag).

The algorithm then proceeds through the observed data, calculating the maximum probability for each state at each step, and maintaining a record of the path that led to that state. Finally, it returns the path with the maximum probability.


In [54]:
# Example inputs
words = ['sos', 'they', 'can', 'fish', 'eos']
tags = ['noun', 'verb']
#start_p = {'noun': 0.5, 'verb': 0.5}
trans_p = {
    'sos': {'noun': 0.37, 'verb': 0.14, 'eos':0},
    'noun': {'noun': 0.05, 'verb': 0.37, 'eos':0.37},
    'verb': {'noun': 0.37, 'verb': 0.05, 'eos':0.37}
}
emit_p = {
    'noun': {'they': 0.14, 'can': 0.05, 'fish': 0.05},
    'verb': {'they': 0.00004, 'can': 0.37, 'fish': 0.05},
}
# Test the Viterbi algorithm
best_sequence_prob, best_sequence = viterbi(words, tags, trans_p, emit_p)

# Print the result
print("Most probable path:", best_sequence)
print("Probability:", best_sequence_prob)



##### Initialization #####

word 1: they

	 current_tag: noun,                                 trans_p['sos'][noun] 0.37,                                 emit_p[noun][they] 0.14,                                 V[1][noun]: 0.051800000000000006
	 current_tag: verb,                                 trans_p['sos'][verb] 0.14,                                 emit_p[verb][they] 4e-05,                                 V[1][verb]: 5.600000000000001e-06

##### Recursion #####

word 2: can
	 current_tag: noun, emit_p 0.05
	 previous_tag: noun                           trans_p[noun]->[noun]: 0.05                           V[1][noun]: 0.051800000000000006                           V[2][noun]:: 0.00012950000000000003
	 previous_tag: verb                           trans_p[verb]->[noun]: 0.37                           V[1][verb]: 5.600000000000001e-06                           V[2][verb]:: 1.0360000000000001e-07
	 max_prob_previous_tag, noun, max_prob: 0.00012950000000000003
	 max V[2][noun] : 0.0001

In [ ]:
import math
math.exp(-2)


0.1353352832366127

In [34]:
list(range(len(words)-1))

[0, 1, 2, 3]